# READING THE DATA

In [43]:
import pandas as pd
train=pd.read_csv("problem_2_train.csv")
train.head()

srno mobmake  hubid  hubpincode  pickuppartnerid  deliverypartnerid  \
0  2851218      AN      7      500004              233                233   
1  2838330      AJ     10      226010              233                233   
2  2835781      AJ      2      110018              233                233   
3  2838589      AM      2      110018              233                233   
4  2855214      AN     21      110015              233                233   

   insuranceCompanyId custpincodecategory claimtype  custpincode  \
0                 131                   A      ADLD       500001   
1                 131                   A      ADLD       226022   
2                 228                   A      ADLD       110018   
3                 228                   B      ADLD       124001   
4                 131                   A      ADLD       201301   

  memcreateddate mobilepurchasedate claimintdate  servicecntrid  \
0     2017-04-27         2016-12-02   2018-02-08           4403   
1     2017-03-06         2017-03-05   2018-02-04           4444   
2     2018-02-01         2018-01-27   2018-02-03           4388   
3     2018-01-17         2018-01-31   2018-02-04           4092   
4     2018-01-23         2018-01-20   2018-02-09           4580   

  pickupStartdate  tat_in_days  
0      2018-02-13            5  
1      2018-02-08           12  
2      2018-02-12            9  
3      2018-02-14            9  
4      2018-02-14            8

# PREPROCESSING 
CONVERTING THE DATES IN THE DATASET TO TIMESTAMPS

In [44]:

import time
import datetime
k=[]
k1=[]
k2=[]
k3=[]
for index,row in train.iterrows():
    s=row["pickupStartdate"]
    row.pickupStartdate=time.mktime(datetime.datetime.strptime(s,"%Y-%m-%d").timetuple())
    k.append(row.pickupStartdate*(10**(-9)))
    s=row["claimintdate"]
    row.claimintdate=time.mktime(datetime.datetime.strptime(s,"%Y-%m-%d").timetuple())
    k1.append(row.claimintdate*(10**(-9)))
    s=row["mobilepurchasedate"]
    row.mobilepurchasedate=time.mktime(datetime.datetime.strptime(s,"%Y-%m-%d").timetuple())
    k2.append(row.mobilepurchasedate*(10**(-9)))
    s=row["memcreateddate"]
    row.memcreateddate=time.mktime(datetime.datetime.strptime(s,"%Y-%m-%d").timetuple())
    k3.append(row.memcreateddate*(10**(-9)))
    #print(row.pickupStartdate)
train["pickupStartdate"]=k
train["claimintdate"]=k1
train["mobilepurchasedate"]=k2
train["memcreateddate"]=k3

adding column diff

In [45]:
k3=[]
for i in range(len(k2)):
    k3.append((k[i]-k1[i])*10000)
train["diff"]=k3

In [46]:
train.head()

srno mobmake  hubid  hubpincode  pickuppartnerid  deliverypartnerid  \
0  2851218      AN      7      500004              233                233   
1  2838330      AJ     10      226010              233                233   
2  2835781      AJ      2      110018              233                233   
3  2838589      AM      2      110018              233                233   
4  2855214      AN     21      110015              233                233   

   insuranceCompanyId custpincodecategory claimtype  custpincode  \
0                 131                   A      ADLD       500001   
1                 131                   A      ADLD       226022   
2                 228                   A      ADLD       110018   
3                 228                   B      ADLD       124001   
4                 131                   A      ADLD       201301   

   memcreateddate  mobilepurchasedate  claimintdate  servicecntrid  \
0        1.493231            1.480617      1.518028           4403   
1        1.488739            1.488652      1.517683           4444   
2        1.517423            1.516991      1.517596           4388   
3        1.516127            1.517337      1.517683           4092   
4        1.516646            1.516387      1.518115           4580   

   pickupStartdate  tat_in_days   diff  
0         1.518460            5  4.320  
1         1.518028           12  3.456  
2         1.518374            9  7.776  
3         1.518547            9  8.640  
4         1.518547            8  4.320

In [5]:
train.custpincodecategory.value_counts()

A    4020
B    1346
C     554
0     186
D     144
Name: custpincodecategory, dtype: int64

# ENCODING THE CATEGORICAL DATA

In [47]:
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
train['custpincodecategory']= label_encoder.fit_transform(train['custpincodecategory'])
train['mobmake']= label_encoder.fit_transform(train['mobmake'])
train['claimtype']= label_encoder.fit_transform(train['claimtype'])

In [7]:
train.head()
train.srno.value_counts()

2951287    4
2865502    4
2885668    3
3038517    3
2956372    2
2891966    2
3011894    2
3012946    2
3044892    2
2934881    2
2819467    2
3056440    2
2822557    2
2878000    2
3039486    2
2960986    2
3039983    2
2923461    2
2882355    2
3039212    2
3019136    2
3048248    2
3051276    2
3019781    2
2961709    2
2822700    2
3022198    2
3074237    2
3005015    2
3071319    2
          ..
2823520    1
3038561    1
2819426    1
3009891    1
2979172    1
2899301    1
2905420    1
2957692    1
3013961    1
3001671    1
3030317    1
2860334    1
2908697    1
3044515    1
3079473    1
2870579    1
2889012    1
2906118    1
3052856    1
2927931    1
2880828    1
2858301    1
2999614    1
2870591    1
2873437    1
3022145    1
3073710    1
2970112    1
2827590    1
2994662    1
Name: srno, Length: 6147, dtype: int64

# APPLYING LINEAR REGRESSION

In [8]:
feature_cols=["srno","memcreateddate","hubid","mobilepurchasedate","pickupStartdate","claimtype","pickuppartnerid","diff","deliverypartnerid","insuranceCompanyId","servicecntrid","custpincodecategory","mobmake","claimintdate"]
X=train[feature_cols]
y=train.tat_in_days

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [10]:
from sklearn.linear_model import LinearRegression
linreg=LinearRegression()
linreg.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
y_pred=linreg.predict(X_test)
print(y_test,y_pred)

1163     9
2743    11
4625    23
5569    11
4948    14
4426     9
2478    20
5509     6
5501     9
3847    10
5134    16
4428     8
4754    13
3100    19
5243    31
3909    13
1817    15
4694    15
2116    10
1697     8
3554     6
3305    23
5343     6
5473     6
2040    17
4447    13
6197    10
1929    21
5000    12
6011     8
        ..
5505     8
4096    16
1077    11
4589    13
1807    14
3085    20
2600     8
3189    21
6109    19
3924    16
900      8
1386    10
5240    12
4716    13
4352    23
5356    13
1357     7
3234     8
16      12
4183    11
2534     6
2754    14
4656    14
761      8
2537    10
1061     9
4195    18
5992    10
3078    12
3525     7
Name: tat_in_days, Length: 1563, dtype: int64 [13.79734278 10.20484625 14.38094514 ... 12.60433077 12.26738483
 11.73290559]


In [12]:
import numpy as np
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,y_pred))

4.989999917605614

In [13]:
linreg.coef_

array([ 3.91268908e-06,  1.06330623e+01,  1.60912430e-03, -1.67857320e+01,
       -3.10766788e+02,  1.54492163e+01, -7.38889274e-03,  8.12314406e-02,
       -9.18942152e-03, -2.88248163e-03, -3.60481455e-03,  1.04610206e+00,
        1.18183554e-01, -3.10766796e+02])

# APPLYING KNN

In [14]:
from sklearn import neighbors
n_neighbors=5
knn=neighbors.KNeighborsRegressor(n_neighbors,weights='uniform')
knn.fit(X_train,y_train)
y_pred=knn.predict(X_test)

In [15]:
import numpy as np
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_test,y_pred))

5.404623503476576

# APPLYING RIDGE

In [16]:
from sklearn import tree
from sklearn import linear_model
model=linear_model.Ridge()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [17]:
np.sqrt(mean_squared_error(y_test,y_pred))

4.989180531443785

In [18]:
train.columns

Index(['srno', 'mobmake', 'hubid', 'hubpincode', 'pickuppartnerid',
       'deliverypartnerid', 'insuranceCompanyId', 'custpincodecategory',
       'claimtype', 'custpincode', 'memcreateddate', 'mobilepurchasedate',
       'claimintdate', 'servicecntrid', 'pickupStartdate', 'tat_in_days',
       'diff'],
      dtype='object')

# REMOVING COLUMNS srno mobmake hubpincode claimtype
# and applying svm

In [19]:
feature_cols=["hubid","memcreateddate","insuranceCompanyId","servicecntrid","custpincodecategory","pickuppartnerid","mobilepurchasedate","deliverypartnerid",'pickupStartdate',"claimtype","claimintdate"]
X=train[feature_cols]
y=train.tat_in_days
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=101)

In [20]:
from sklearn import svm
svm_reg=svm.SVR()
svm_reg.fit(X_train,y_train)
y_pred=svm_reg.predict(X_test)

In [21]:
np.sqrt(mean_squared_error(y_test,y_pred))

4.5822974190613115

# applying descision tree

In [22]:
feature_cols=["hubid","memcreateddate","insuranceCompanyId","servicecntrid","custpincodecategory","pickuppartnerid","mobilepurchasedate","deliverypartnerid",'pickupStartdate',"claimtype","claimintdate"]
X=train[feature_cols]
y=train.tat_in_days
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [23]:
dec = tree.DecisionTreeRegressor(max_depth=1)
dec.fit(X_train,y_train)
y_pred=dec.predict(X_test)

In [24]:
np.sqrt(mean_squared_error(y_test,y_pred))

5.177018440869705

# applying gradientboost

In [63]:
feature_cols=["memcreateddate","insuranceCompanyId","servicecntrid","custpincodecategory","pickuppartnerid","mobilepurchasedate","deliverypartnerid",'pickupStartdate',"claimtype","claimintdate"]
X=train[feature_cols]
y=train.tat_in_days
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=101 )

In [64]:
from sklearn.ensemble import GradientBoostingRegressor
gb=GradientBoostingRegressor(n_estimators =200)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [65]:
np.sqrt(mean_squared_error(y_test,y_pred))

4.161820943312321

# appyling random forest

In [40]:
feature_cols=["memcreateddate","insuranceCompanyId","servicecntrid","custpincodecategory","pickuppartnerid","mobilepurchasedate","deliverypartnerid",'pickupStartdate',"claimtype","claimintdate"]
X=train[feature_cols]
y=train.tat_in_days
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [41]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=5,
                              n_estimators=100)
regr.fit(X_train,y_train)
y_pred=regr.predict(X_test)

In [42]:
np.sqrt(mean_squared_error(y_test,y_pred))

4.723346018102905

# saving to csv using gradient boost min rms=4.16

In [28]:
test=pd.read_csv("problem_2_test.csv")
test.head()

srno mobmake  hubid  hubpincode  pickuppartnerid  deliverypartnerid  \
0  2829088      AN      3      560011              233                233   
1  2829866      AJ      2      110018              233                233   
2  2894489      AJ      3      560011              233                233   
3  2876708      AJ      2      110018              233                233   
4  2890283      AN     22      400104              233                233   

   insuranceCompanyId custpincodecategory claimtype  custpincode  \
0                 131                   A      ADLD       560093   
1                 228                   A      ADLD       110034   
2                 228                   C      ADLD       590014   
3                 228                   B      ADLD       250110   
4                 131                   C      ADLD       496001   

  memcreateddate mobilepurchasedate claimintdate  servicecntrid  \
0     2017-07-30         2017-07-27   2018-02-01           4467   
1     2017-07-17         2017-01-25   2018-02-01           4388   
2     2018-01-12         2018-01-11   2018-02-21           4460   
3     2017-08-07         2017-05-21   2018-02-16           4388   
4     2017-11-10         2017-09-22   2018-02-20           4581   

  pickupStartdate  
0      2018-02-06  
1      2018-02-08  
2      2018-02-28  
3      2018-02-26  
4      2018-02-23

In [29]:
import time
import datetime
k=[]
k1=[]
k2=[]
k3=[]
for index,row in test.iterrows():
    s=row["pickupStartdate"]
    row.pickupStartdate=time.mktime(datetime.datetime.strptime(s,"%Y-%m-%d").timetuple())
    k.append(row.pickupStartdate*(10**(-9)))
    s=row["claimintdate"]
    row.claimintdate=time.mktime(datetime.datetime.strptime(s,"%Y-%m-%d").timetuple())
    k1.append(row.claimintdate*(10**(-9)))
    s=row["mobilepurchasedate"]
    row.mobilepurchasedate=time.mktime(datetime.datetime.strptime(s,"%Y-%m-%d").timetuple())
    k2.append(row.mobilepurchasedate*(10**(-9)))
    s=row["memcreateddate"]
    row.memcreateddate=time.mktime(datetime.datetime.strptime(s,"%Y-%m-%d").timetuple())
    k3.append(row.memcreateddate*(10**(-9)))
    #print(row.pickupStartdate)
test["pickupStartdate"]=k
test["claimintdate"]=k1
test["mobilepurchasedate"]=k2
test["memcreateddate"]=k3

In [30]:
k3=[]
for i in range(len(k2)):
    k3.append((k[i]-k1[i])*10000)
test["diff"]=k3

In [31]:
test

srno mobmake  hubid  hubpincode  pickuppartnerid  deliverypartnerid  \
0     2829088      AN      3      560011              233                233   
1     2829866      AJ      2      110018              233                233   
2     2894489      AJ      3      560011              233                233   
3     2876708      AJ      2      110018              233                233   
4     2890283      AN     22      400104              233                233   
5     2897438       C      6      160059              233                233   
6     2849010       C      2      110018              233                233   
7     2887783       U      2      110018              233                233   
8     2941799      AN     21      110015              233                233   
9     2973976      AN     22      400104              233                233   
10    2977421       J      2      110018              233                233   
11    2982990      AN     26      560066              233                233   
12    2979785      AJ      2      110018              233                233   
13    2984029       H      1      400059              233                233   
14    3012909      AD      2      110018              233                233   
15    3030307      AJ     27      110034              233                233   
16    3048314      AN      7      500004              233                233   
17    3060524       C      2      110018              233                233   
18    3064259      AN     21      110015              233                233   
19    3057867       C      2      110018              308                233   
20    2870652      AJ      3      560011              233                245   
21    2920134      AJ      3      560011              233                245   
22    2866111      AN     21      110015              233                233   
23    2908430      AJ      3      560011              233                233   
24    2920849      AN     22      400104              233                233   
25    2884266      AF      5      380009              233                245   
26    2819466      AN      4      600024              233                233   
27    2858437      AN     17      800001              233                233   
28    3050523       C      2      110018              308                233   
29    2876140      AN     22      400104              233                245   
...       ...     ...    ...         ...              ...                ...   
2086  2973530      AN     26      560066              233                233   
2087  3031983      AN     22      400104              233                233   
2088  2841201       C      2      110018              233                233   
2089  2906715      AN      3      560011              233                233   
2090  2990607      AJ     25      560037              201                233   
2091  2843872       C     12      302006              233                233   
2092  2889644      AJ      2      110018              233                233   
2093  2906784      AN      8      700064              233                233   
2094  2891769       C     12      302006              201                233   
2095  2898703      AN      3      560011              233                233   
2096  2910059       J      2      110018              233                233   
2097  2953259      AJ      2      110018              201                233   
2098  2947954      AN     21      110015              233                233   
2099  2956245      AI      6      160059              233                233   
2100  2931393      AN     21      110015              308                233   
2101  3039774       C      2      110018              233                233   
2102  3074175      AN      7      500004              233                233   
2103  3075677      AJ     23      400001              233                233   
2104  298

In [32]:
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
test['custpincodecategory']= label_encoder.fit_transform(test['custpincodecategory'])
test['mobmake']= label_encoder.fit_transform(test['mobmake'])
test['claimtype']= label_encoder.fit_transform(test['claimtype'])

In [33]:
len(test)

2116

In [34]:
feature_cols=["memcreateddate","insuranceCompanyId","servicecntrid","custpincodecategory","pickuppartnerid","mobilepurchasedate","deliverypartnerid",'pickupStartdate',"claimtype","claimintdate"]
X_test=test[feature_cols]

In [35]:
len(X_test)

2116

In [36]:
y_pred=gb.predict(X_test)

In [37]:
len(y_pred)

2116

In [38]:
res=pd.DataFrame({'tat_in_days':y_pred})

In [39]:
res.to_csv("solutions.csv")